In [ ]:
import torch
import numpy as np

In [ ]:
def knn(data, k=5)->torch.Tensor:
    """Construct edge feature for each point
    Args:
      point_cloud: (batch_size, num_points, num_dims)
      k: int number of neighbours

    Returns:
      idx: shape:(batch_size, num_points, nums_neighours,)
    """
    dists_matrix = torch.cdist(data, data)
    #print(dists_matrix.shape)
    _, idx = dists_matrix.topk(k+1, dim=-1, largest=False)  # +1 the point itself is included
    return idx[...,1:] # not include the point itself



def get_edge_feature(point_cloud, idx=None, k=20,device="cpu"):
    """Construct edge feature for each point
    Args:
      point_cloud: (batch_size, num_points, num_dims)
      idx: (batch_size, num_points, neighbours)
      k: int
      device: cpu/cuda

    Returns:
      features: (batch_size, num_dims ,num_points, k)
    """
    point_cloud = point_cloud.to(device)
    batch_size = point_cloud.shape[0]
    num_points = point_cloud.shape[1]

    if(idx==None):
        idx = knn(point_cloud,k=k) # (batch_size, num_points, nums_neighours,)

    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1) * num_points # create the base index for mapping
    idx = idx.to(device=device)
    idx = idx + idx_base #[0...0...0]->[0...100...200]
    idx=idx.view(-1) # flatten it -> tensor([  0,  56,  25,  ..., 225, 222, 271], device='cuda:0') e.g: [K01,K02,K03,K11,K12,K13...] shape = (B*N*K) 
   
    num_dims = point_cloud.shape[2]

    # feature : turn neighbour index in idx to coordinate
    feature = point_cloud.view(batch_size*num_points, -1)[idx, :] # feature : B*N*F -> BN * F -> (B*N*K) * F
    # feature : reshape into (Batch_size * Num_points *Nums_neigbours * Features)
    feature = feature.view(batch_size, num_points, k, num_dims)
    
    # pointcloud : create replicate of the self point up to k for matching feature - size B*N*K(repeated)*F 
    point_cloud = point_cloud.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1) 

    # feature size B*N*K*F -> B*N*K*2F (feature-x || x)
    feature = torch.cat((feature-point_cloud, point_cloud), dim=3)

    # (B * 2F * N * K) for later conv each coordinate(F)
    feature=feature.permute(0,3,1,2).contiguous()

    return feature

# Example usage:
data = torch.rand((3,100, 3))  # 100 points in 20D (batch_size, num_points, num_dims)
#neighbors = knn(data, k=4)
edges= get_edge_feature(data)
print(edges.shape)
print(type(edges))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EdgeConv(nn.Module):
    def __init__(self, in_channels, out_channels, num_neighbours=20,device="cpu"):
        """Setup EdgeConv
        Args:
        in_channels: int
        out_channels: int
        num_neighbours: int
        """
        super(EdgeConv, self).__init__()
        self.device=device
        self.k= num_neighbours
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=in_channels*2, out_channels=out_channels, kernel_size=1, bias=False,device=self.device),
            nn.BatchNorm2d(out_channels,device=self.device),
            nn.LeakyReLU(negative_slope=0.2)
        )

    def forward(self,x):
        """Setup EdgeConv
        Args:
        x: shape - (batch_size, num_points, num_dims)

        Returns:
        features: (batch_size, num_dims, num_points, num_neigbours)
        """
        x = get_edge_feature(x, k=self.k,device=self.device) #(batch_size, num_points, dim) -> (batch_size, dim*2, num_points ,k)
        x = self.conv(x)
        # for each point pick the largest k (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)
        x = x.max(dim=-1, keepdim=False)[0]
        x = x.permute(0,2,1).contiguous()
        return x
    
# Example usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = torch.rand((3,100, 3))  # 100 points in 20D (batch_size, num_points, num_dims)
conv = EdgeConv(3, 64,device=device)
out = conv(data)
print("out.shape=", out.shape)

In [ ]:
class DGCNN(nn.Module):
    def __init__(self, num_neighbours=20,out_channels=40,dropout_rate =0.3,device="cpu"):
        super(DGCNN,self).__init__()
        self.inChannels=[3,64,64,128,256]
        self.edgeConv0 = EdgeConv(in_channels=3,out_channels=64,num_neighbours=num_neighbours,device=device)
        self.edgeConv1 = EdgeConv(in_channels=64,out_channels=64,num_neighbours=num_neighbours,device=device)
        self.edgeConv2 = EdgeConv(in_channels=64,out_channels=128,num_neighbours=num_neighbours,device=device)
        self.edgeConv3 = EdgeConv(in_channels=128,out_channels=256,num_neighbours=num_neighbours,device=device)

        self.edgeConv4 = EdgeConv(in_channels=512,out_channels=1024,num_neighbours=num_neighbours,device=device)

        self.linear1 = nn.Linear(2048, 512, bias=False,device=device)
        self.bn1 = nn.BatchNorm1d(512,device=device)
        self.drop1 = nn.Dropout(dropout_rate)
        self.linear2 = nn.Linear(512, 256, bias=False,device=device)
        self.bn2 = nn.BatchNorm1d(256,device=device)
        self.drop2 = nn.Dropout(dropout_rate)
        self.linear3 = nn.Linear(256,out_channels, bias=False,device=device)


    def forward(self,x):
        x0=self.edgeConv0(x)
        #print("x0:",x0.shape)
        x1=self.edgeConv1(x0)
        x2=self.edgeConv2(x1)
        x3=self.edgeConv3(x2)

        x=torch.cat((x0,x1,x2,x3),dim=2)
        
        x= self.edgeConv4(x) # (batch_size, num_points ,64+64+128+256) -> (batch_size, num_points, emb_dims(1024))
        
        #todo 
        # maxpool and avgpool
        x= x.permute(0,2,1).contiguous()
        maxPoolX = F.adaptive_avg_pool1d(x,1).view(x.shape[0],-1)
        avgPoolX = F.adaptive_avg_pool1d(x,1).view(x.shape[0],-1)
        x=torch.cat((maxPoolX,avgPoolX),1) #(batch_size, 2048)
        
        #mlp[512,256,c(40)]
        x= F.leaky_relu(self.bn1(self.linear1(x)))
        x=self.drop1(x)
        x= F.leaky_relu(self.bn2(self.linear2(x)))
        x=self.drop2(x)
        x= self.linear3(x)
        # output

        return x
    
# Example usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = torch.rand((3,100, 3))  # 100 points in 20D (batch_size, num_points, num_dims)
print("data shape: ", data.shape)
print(device)
dgcnn = DGCNN(device=device)
out = dgcnn(data)

print("out.shape=", out.shape)

In [ ]:
class DGCNN_S(nn.Module):
    def __init__(self, num_neighbours=10,out_channels=40,device="cpu"):
        super(DGCNN_S,self).__init__()
        self.inChannels=[3,64,64,128,256]
        self.edgeConv0 = EdgeConv(in_channels=3,out_channels=64,num_neighbours=num_neighbours,device=device)
        self.edgeConv1 = EdgeConv(in_channels=64,out_channels=64,num_neighbours=num_neighbours,device=device)
        self.edgeConv2 = EdgeConv(in_channels=64,out_channels=128,num_neighbours=num_neighbours,device=device)

        self.edgeConv4 = EdgeConv(in_channels=256,out_channels=512,num_neighbours=num_neighbours,device=device)

        self.linear1 = nn.Linear(512, 256, bias=False,device=device)
        self.bn1 = nn.BatchNorm1d(256,device=device)
        self.linear2 = nn.Linear(256,out_channels, bias=False,device=device)


    def forward(self,x):
        x0=self.edgeConv0(x)
        #print("x0:",x0.shape)
        x1=self.edgeConv1(x0)
        x2=self.edgeConv2(x1)

        x=torch.cat((x0,x1,x2),dim=2)
        
        x= self.edgeConv4(x) # (batch_size, num_points ,64+64+128+256) -> (batch_size, num_points, emb_dims(1024))
        
        #todo 
        # maxpool and avgpool
        x= x.permute(0,2,1).contiguous()
        x = F.adaptive_avg_pool1d(x,1).view(x.shape[0],-1)
        
        #mlp[512,256,c(40)]
        x= F.leaky_relu(self.bn1(self.linear1(x)))
        x= self.linear2(x)
        # output

        return x
    
# Example usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = torch.rand((3, 100, 3))  # 100 points in 20D (batch_size, num_points, num_dims)
print("data shape: ", data.shape)
print(device)
dgcnn = DGCNN_S(device=device)
out = dgcnn(data)

print("out.shape=", out.shape)

In [ ]:
import os
import glob
import h5py
import numpy as np
from torch.utils.data import Dataset

def download_modelnet40():
    DATA_DIR = 'data'
    if not os.path.exists(DATA_DIR):
        os.mkdir(DATA_DIR)
        www = 'https://shapenet.cs.stanford.edu/media/modelnet40_ply_hdf5_2048.zip'
        zipfile = os.path.basename(www)
        os.system('curl -k -O %s; unzip %s' % (www, zipfile))
        os.system('mv %s %s' % ('modelnet40_ply_hdf5_2048', DATA_DIR))
        os.system('rm %s' % (zipfile))
def download_modelnet40():
    BASE_DIR = os.getcwd()
    DATA_DIR = os.path.join(BASE_DIR, 'data')
    print(os.path.exists(os.path.join(DATA_DIR, 'modelnet40_ply_hdf5_2048')))
    if not os.path.exists(DATA_DIR):
        os.mkdir(DATA_DIR)
    if not os.path.exists(os.path.join(DATA_DIR, 'modelnet40_ply_hdf5_2048')):
        www = 'https://shapenet.cs.stanford.edu/media/modelnet40_ply_hdf5_2048.zip'
        zipfile = os.path.basename(www)
        os.system('wget --no-verbose --no-check-certificate %s; unzip %s' % (www, zipfile))
        os.system('mv %s %s' % ('modelnet40_ply_hdf5_2048', DATA_DIR))
        os.system('rm %s' % (zipfile))

# def load_data_cls(partition):
#     download_modelnet40()
#     BASE_DIR = os.getcwd()
#     DATA_DIR = os.path.join(BASE_DIR, 'data')
#     all_data = []
#     all_label = []
#     for h5_name in glob.glob(os.path.join(DATA_DIR, 'modelnet40_ply_hdf5_2048', '*%s*.h5'%partition)):
#         f = h5py.File(h5_name, 'r+')
#         data = f['data'][:].astype('float32')
#         label = f['label'][:].astype('int64')
#         f.close()
#         all_data.append(data)
#         all_label.append(label)
#     all_data = np.concatenate(all_data, axis=0)
#     all_label = np.concatenate(all_label, axis=0)
#     return all_data, all_label

# def translate_pointcloud(pointcloud):
#     xyz1 = np.random.uniform(low=2./3., high=3./2., size=[3])
#     xyz2 = np.random.uniform(low=-0.2, high=0.2, size=[3])
       
#     translated_pointcloud = np.add(np.multiply(pointcloud, xyz1), xyz2).astype('float32')
#     return translated_pointcloud
def load_data_cls():
    download_modelnet40()
    BASE_DIR = os.getcwd()
    DATA_DIR = os.path.join(BASE_DIR, 'data')
    all_data = []
    all_label = []
    for h5_name in glob.glob(os.path.join(DATA_DIR, 'modelnet40_ply_hdf5_2048', '*.h5')):
        f = h5py.File(h5_name, 'r+')
        data = f['data'][:].astype('float32')
        label = f['label'][:].astype('int64')
        f.close()
        all_data.append(data)
        all_label.append(label)
    all_data = np.concatenate(all_data, axis=0)
    all_label = np.concatenate(all_label, axis=0)
    return all_data, all_label

def translate_pointcloud(pointcloud):
    xyz1 = np.random.uniform(low=2./3., high=3./2., size=[3])
    xyz2 = np.random.uniform(low=-0.2, high=0.2, size=[3])
       
    translated_pointcloud = np.add(np.multiply(pointcloud, xyz1), xyz2).astype('float32')
    return translated_pointcloud

class ModelNet40(Dataset):
#     def __init__(self, num_points, partition='train'):
#         self.data, self.label = load_data_cls(partition)
#         self.num_points = num_points
#         self.partition = partition        

#     def __getitem__(self, item):
#         pointcloud = self.data[item][:self.num_points]
#         label = self.label[item]
#         if self.partition == 'train':
#             pointcloud = translate_pointcloud(pointcloud)
#             np.random.shuffle(pointcloud)
#         return pointcloud, label

#     def __len__(self):
#         return self.data.shape[0]
    def __init__(self, num_points, use_whole_dataset=True):
        self.data, self.label = load_data_cls()
        self.num_points = num_points
        self.use_whole_dataset = use_whole_dataset

    def __getitem__(self, item):
        pointcloud = self.data[item][:self.num_points]
        label = self.label[item]
        if not self.use_whole_dataset:
            pointcloud = translate_pointcloud(pointcloud)
            np.random.shuffle(pointcloud)
        return pointcloud, label

    def __len__(self):
        return self.data.shape[0]
    
# use like this
data = ModelNet40(1024)
_data, label = data[0]
print(_data.shape)
print(label.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# PATH = "/kaggle/input/model2/model.t7"
# lr = 0.001
# num_epochs = 50
# num_points = 1024
# batch_size = 8
# temperature = 2.0
# apha = 0.5
# beta = 0.5

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# train_loader = DataLoader(ModelNet40(num_points, 'train'), batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(ModelNet40(num_points, 'test'), batch_size=batch_size, shuffle=False)
# teacher_model = nn.DataParallel(DGCNN(device=device))
# teacher_model.load_state_dict(torch.load(PATH,map_location=device))
# student_model = nn.DataParallel(DGCNN_S(device=device))
# optimizer = optim.Adam(student_model.parameters(), lr)
# distillation_loss = KnowledgeDistillationLoss(temperature=temperature)

# best_test_acc = 0
# for epoch in range(num_epochs):
#     #train
#     student_model.train()
#     total_loss = 0
#     for data, label in train_loader:
#         data, label = data.to(device), label.to(device).squeeze()
#         optimizer.zero_grad()
#         student_logits = student_model(data)
#         teacher_logits = teacher_model(data)
#         loss = apha * distillation_loss(student_logits, teacher_logits) + beta * F.cross_entropy(student_logits, label)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {total_loss / len(train_loader):.4f}')
#     #test
#     student_model.eval()
#     total = 0
#     correct = 0
#     total_loss = 0
#     with torch.no_grad():
#         for data, label in test_loader:
#             data, label = data.to(device), label.to(device).squeeze()
#             output = student_model(data)
#             loss = F.cross_entropy(output, label)
#             total_loss += loss.item()
#             _, predicted = output.max(1)
#             total += label.size(0)
#             correct += predicted.eq(label).sum().item()
#     test_acc = correct / total
#     print(f'Epoch {epoch+1}/{num_epochs}, Test Loss: {total_loss / len(test_loader):.4f}, Test Acc: {test_acc:.4f}')
#     if test_acc >= best_test_acc:
#         best_test_acc = test_acc
#         torch.save(student_model.state_dict(), '/kaggle/working/student_model_50.t7')
PATH_teacher = "/kaggle/input/model4/model.t7"
PATH_student="/kaggle/input/model4/student_model_50.t7"
num_points = 1024
batch_size = 8

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_loader = DataLoader(ModelNet40(num_points), batch_size=batch_size, shuffle=False)
# teacher_model = nn.DataParallel(DGCNN(device=device))
# teacher_model.load_state_dict(torch.load(PATH_teacher,map_location=device))
# teacher_model.to(device)
# teacher_model.eval()
student_model = nn.DataParallel(DGCNN_S(device=device))
student_model.load_state_dict(torch.load(PATH_student, map_location=device))
student_model.to(device)
student_model.eval()

total = 0
correct = 0
total_loss = 0

# with torch.no_grad():
#     for data, label in test_loader:
# #         print("test")
#         data, label = data.to(device), label.to(device).squeeze()
#         output = student_model(data)
#         loss = F.cross_entropy(output, label)
#         total_loss += loss.item()
#         _, predicted = output.max(1)
#         total += label.size(0)
#         correct += predicted.eq(label).sum().item()

# test_acc = correct / total
# print(f'Test Loss: {total_loss / len(test_loader):.4f}, Test Acc: {test_acc:.4f}')
num_classes=40
class_correct = [0] * num_classes  # Used to record the correct number of categories for each category
class_total = [0] * num_classes  # Used to record the total number of samples for each category

with torch.no_grad():
    for data, label in test_loader:
        data, label = data.to(device), label.to(device).squeeze()
        output = student_model(data)
        loss = F.cross_entropy(output, label)
        total_loss += loss.item()
        _, predicted = output.max(1)
        total += label.size(0)
        correct += predicted.eq(label).sum().item()

        
        for i in range(label.size(0)):
            class_correct[label[i]] += predicted[i].eq(label[i]).item()
            class_total[label[i]] += 1

test_acc = correct / total
average_per_class_acc = sum(class_correct[i] / class_total[i] for i in range(num_classes)) / num_classes
overall_acc = sum(class_correct) / sum(class_total)

print(f'Test Loss: {total_loss / len(test_loader):.4f}')
print(f'Test Acc: {test_acc:.4f}')
print(f'Average Per-Class Acc: {average_per_class_acc:.4f}')
print(f'Overall Acc: {overall_acc:.4f}')